In [48]:
import sys
import json
import re
import pandas as pd
import pickle
import numpy as np

In [5]:
lib_hashtags = set([ht.lower() for ht in [
    'resist',
    'impeachtrump',
    'p2',
    'bluewave2018'
    'fucktrump',
    'resistance',
    'bluewave',
    'fbr', 
    'flipitblue',
    'votebluetosaveamerica',
    'votewithbeto', 
    'Votebeto',
    'betofortexas',
    'gopvotingblue',
    'resisttrump',
    'iamawakenow', 
    'votethemout2018',
    'votebluetoendthisnightmare',
    'trumplies',
    'trumpliesmatter', 
    'goplies',
    'factsmatter',
    'flipthe49th',
    'VoteScholten4Iowa',
    'thisisnotnormal',
    'takeitback',
    'blacklivesmatter',
    'dumptrump',
    'antitrump',
    'protectourcare',
    'votedem',
    'FlipTheHouse',
    'FlipTheSenate',
    'RejectTheNRA',
    'VoteBlue',
    'BlueWave2018',
    'GunSense',
    'flipthe45th',
    'VoteBlueToSaveHealthcare',
    'OpenBorders',
    'VoteBlue2018',
    'CABlueWAVE',
    'VoteBlueThisTuesday',
    'VoteBLUENoMatterWho',
    'Beto4Texas',
    'VoteBlueToSaveAmerica',
    'VoterSuppressionTactics',
    'AbolishIce',
    'Bernie2020',
    'VoterSuppression',
    'VoteForTimCanova',
    'VoteBlueToSaveAmericanDemocracy',
    'VoteJanz',
    'VoteClaire',
    'VoteforAndrewJanz',
    'ReplaceNunes',
    'RepublicanLiars',
    'ResistandWin',
    'medicareforall',
    'freecollege',
    'ObamaLegacy',
    'DemsSoDiverse',
    'FLIPITVOTEBLUE',
    'PreExistingConditions',
    'goodtrouble',
    'REPUBLICUNT',
    'BoycottTysonFoods',
    'StaceyAbrams4GAgov',
    'GunSenseCandidate',
    'TeamAbrams',
    'VoteBlueToSaveAmerica',
    'medicare4all',
    'neonazi',
    'neonazis',
    'whitesupremacist',
    'whitesupremacists',
    'turngablue',
    'prochoice',
    'VoteScholten4Iowa',
    'VoteWalkerOut',
    'LGBT',
    'lgbtq',
    'lgbtq+',
    'GOPTaxScam',
    'LGBTQIA',
    'VoteBlueToSaveDemocracy',
    'VoteGOPOut',
    'StandIndivisible',
    'FlipThe8th',
    'waveCast',
    'WaveCastWA',
    'BuildTheBlueWave',
    '14yearsisenough',
    'flipthe3rd',
    'flipca25',
    'VoteBetoORourke4Texas',
    'ResistanceRises',
    'feelthebern',
]])
cons_hashtags = set([ht.lower() for ht in [
    'trumptrain',
    'maga',
    'walkaway',
    'qanon',
    'changethemedia',
    'greatawakening',
    'americafirst',
    'wwg1wga',
    'tcot',
    'kag',
    'patriotsunited',
    'qarmy',
    'ccot',
    'thegreatawakening',
    'q',
    'trump2020',
    'makeamericagreatagain',
    'liberalismisamentaldisorder',
    'trumpsarmy',
    'voteredtosaveamerica',
    'votered',
    'unitedvotered',
    'supportice',
    'keepredstatesred',
    'turnbluestatesred',
    'voteredtosavecalifornia',
    'MadMaxine',
    'VoteRedToSaveAmerica2018',
    'ResistTheResistance',
    'VoteAntonioToSaveCalifornia',
    'VoteRed2018',
    'GetMaxineOut',
    'RedWave2018',
    'blexit',
    'PledgeToVoteRed',
    'MAGA2020',
    'VoteRedSaveAmerica2018',
    'RedWave2018andBeyond',
    'redcalifornia',
    'VoteDemsOut',
    'MakeAmericsGreatAgain',
    'REDWAVECOMING',
    'RedTsunami',
    'VoteAntonioSabato',
    'VoteRedOrAmericaIsDead',
    'RedWave',
    'VoteRedSaveAmerica',
    'VoteRedMidterms2018',
    'CaravanInvasion',
    'WalkAwayFromDemocrats',
    'RedNovemeber',
    'RedNovember',
    'KeepFloridaRed',
    'BuildTheWall',
    'DemocratsHateAmerica',
    'VoteAntonio2018',
    'IVotedRED',
    'DefundPlannedParenthood',
    'MSM',
    'VoteRedToSaveCalifornia',
    'LyingLiberals',
    'UnhingedLeft',
    'CHOOSECRUZ',
    'RedTsunamiNov6',
    'whitepower',
    'prolife',
    'TeamDino',
    'waelex',
    'wapol',
    'betteroffnow',
    'votejaime',
    'wapol',
    'gregformontana',
    'teamgiantforte',
    'watergrab',
    'deandelivers',
    'AngryDemocratMob',
    'bluelivesmatter'
]])


In [6]:
infile = 'candidate_handles3B2018-11-02.json'

In [7]:
with open('candidate_handles.txt') as f:
    partisan_tweets_per_candidate = {}
    handles = []
    for line in f:
        partisan_tweets_per_candidate[line[:-1].lower()] = {'liberal': 0, 'conservative': 0, 'neutral': 0}
        handles.append(line[:-1].lower())

In [8]:
blue_pattern = re.compile(".*blue*")
bool(blue_pattern.match('voteblue'))

True

In [33]:
sys.stdout.write("parsing " + infile + "...")
sys.stdout.flush()

with open(infile) as f:
    blue_pattern = re.compile(".*blue*")
    red_pattern = re.compile(".*red*")
    for line in f:
        if line != None:
            tweet = json.loads(line.rstrip('\n'))
            try:
                # Classify tweet as liberal or conservative
                lib_score = 0
                cons_score = 0
                tweet_sentiment = 'neutral'
                for hashtag in tweet['entities']['hashtags']:
                    ht = hashtag['text'].lower()
                    if ht in lib_hashtags or blue_pattern.match(ht):
                        lib_score +=1
                    elif ht in cons_hashtags or red_pattern.match(ht):
                        cons_score += 1
                if lib_score > cons_score:
                    tweet_sentiment = 'liberal'
#                         print(tweet['text'].lower())
                elif lib_score < cons_score:
                    tweet_sentiment = 'conservative'
                else:
                    if tweet['entities']['hashtags']:
                        print(tweet['entities']['hashtags'])

                # Get the candidates mentioned in tweets
                mentions = re.findall('@[a-zA-Z0-9_]{1,15}', tweet['text'].lower())
                # Update dictionary
                for person in mentions:
                    try:
                        partisan_tweets_per_candidate[person][tweet_sentiment] += 1
                    except:
                        pass;
#                     print(tweet_sentiment)
            except:
                print('FAILED')


parsing candidate_handles3B2018-11-02.json...

FileNotFoundError: [Errno 2] No such file or directory: 'candidate_handles3B2018-11-02.json'

In [34]:
partisan_tweets_per_candidate

{'@kennedy4alabama': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repbyrne': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@tabithak': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repmartharoby': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@haganforhouse': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repmikerogersal': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@leeauman2018': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@robert_aderholt': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@peter_joffrion': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repmobrooks': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@votedannerkline': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@usrepgarypalmer': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repterrisewell': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repdonyoung': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@alysegalvin': {'conservative': 0, 'liberal': 0, 

## Predicting election outcomes

We have a map from candidate name to Twitter handle, and the csv with district, state, chamber, party affiliation, and incumbency data for each candidate. We want a dataframe with district, state, chamber, projected winner's name, party, incumbency for social influence model, and projected winner's name, party, incumbency for voter count model.

input: dictionary that maps name to handle

In [3]:
candidate_data = pd.read_csv('../candidate_data_final.csv')

In [4]:
candidate_data

,district,name,state,chamber,party,incumbent
0,District 1,Robert Kennedy Jr.,Alabama,house,Democratic Party,False
1,District 1,Bradley Byrne,Alabama,house,Republican Party,True
2,District 2,Tabitha Isner,Alabama,house,Democratic Party,False
3,District 2,Martha Roby,Alabama,house,Republican Party,True
4,District 3,Mallory Hagan,Alabama,house,Democratic Party,False
5,District 3,Mike Rogers,Alabama,house,Alabama Party,True
6,District 4,Lee Auman,Alabama,house,Democratic Party,False
7,District 4,Robert Aderholt,Alabama,house,Republican Party,True
8,District 5,Peter Joffrion,Alabama,house,Democratic Party,False
9,District 5,Mo Brooks,Alabama,house,Republican Party,True


In [5]:
with open('candidate_names_to_handles.pkl', 'rb') as f:
    candidate_names_to_handles = pickle.load(f)

In [6]:
candidate_data

,district,name,state,chamber,party,incumbent
0,District 1,Robert Kennedy Jr.,Alabama,house,Democratic Party,False
1,District 1,Bradley Byrne,Alabama,house,Republican Party,True
2,District 2,Tabitha Isner,Alabama,house,Democratic Party,False
3,District 2,Martha Roby,Alabama,house,Republican Party,True
4,District 3,Mallory Hagan,Alabama,house,Democratic Party,False
5,District 3,Mike Rogers,Alabama,house,Alabama Party,True
6,District 4,Lee Auman,Alabama,house,Democratic Party,False
7,District 4,Robert Aderholt,Alabama,house,Republican Party,True
8,District 5,Peter Joffrion,Alabama,house,Democratic Party,False
9,District 5,Mo Brooks,Alabama,house,Republican Party,True


In [7]:
len(candidate_names_to_handles)

1180

In [8]:
candidate_names_to_handles['Robert Kennedy Jr.']

'Kennedy4Alabama'

In [9]:
def map_names_to_handles(row):
    try:
        return candidate_names_to_handles[row['name']].lower()
    except:
        pass

In [10]:
candidate_data['handles'] = candidate_data.apply(map_names_to_handles, axis=1)

In [11]:
candidate_data[candidate_data['handles'] == None]

,district,name,state,chamber,party,incumbent,handles


In [12]:
len(candidate_data[candidate_data["handles"].isnull()])

61

In [13]:
races = candidate_data.astype(str).groupby(['district', 'state', 'chamber'])
races

In [14]:
list((5,4))

[5, 4]

In [15]:
len(races)

439

In [16]:
len(races.groups)

439

In [17]:
races.groups

{('District 1', 'Alabama', 'house'): Int64Index([0, 1], dtype='int64'),
 ('District 1', 'Arizona', 'house'): Int64Index([17, 18, 19], dtype='int64'),
 ('District 1', 'Arkansas', 'house'): Int64Index([41, 42, 43], dtype='int64'),
 ('District 1', 'California', 'house'): Int64Index([53, 54], dtype='int64'),
 ('District 1',
  'Colorado',
  'house'): Int64Index([158, 159, 160, 161], dtype='int64'),
 ('District 1',
  'Connecticut',
  'house'): Int64Index([193, 194, 195], dtype='int64'),
 ('District 1', 'Georgia', 'house'): Int64Index([270, 271], dtype='int64'),
 ('District 1',
  'Hawaii',
  'house'): Int64Index([304, 305, 306, 307, 308], dtype='int64'),
 ('District 1',
  'Idaho',
  'house'): Int64Index([311, 312, 313, 314, 315, 316, 317, 318], dtype='int64'),
 ('District 1',
  'Illinois',
  'house'): Int64Index([321, 322, 323], dtype='int64'),
 ('District 1',
  'Indiana',
  'house'): Int64Index([363, 364, 365], dtype='int64'),
 ('District 1', 'Iowa', 'house'): Int64Index([386, 387, 388], dty

In [18]:
# for race in races.groups.keys():
#     print(race)

In [58]:
new_df = pd.DataFrame(columns=['district','state', 'chamber', 'winner_predicted_by_total', 'winner_predicted_by_unique'])
i = 0
for race, df in races:
    predictions = get_winner(df, partisan_tweets_per_candidate)
    print(predictions)
    new_df = new_df.append({
        'district': race[0], 
        'state': race[1], 
        'chamber': race[2], 
        'winner_predicted_by_total': predictions['winner_predicted_by_total'],
        'margin_total': predictions['margin_total'],
        'winner_predicted_by_unique': predictions['winner_predicted_by_unique'],
        'margin_unique': predictions['margin_unique']
    }, ignore_index = True)
    i += 1

{'winner_predicted_by_total': 'Robert Kennedy Jr.', 'margin_total': 0.28, 'winner_predicted_by_unique': 'Robert Kennedy Jr.', 'margin_unique': 0.09090909090909091}
{'winner_predicted_by_total': 'Wendy Rogers', 'margin_total': nan, 'winner_predicted_by_unique': 'Wendy Rogers', 'margin_unique': nan}
{'winner_predicted_by_total': 'Rick Crawford', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Elvis Presley', 'margin_unique': nan}
{'winner_predicted_by_total': 'Audrey Denney', 'margin_total': 0.8913043478260869, 'winner_predicted_by_unique': 'Audrey Denney', 'margin_unique': 0.9433962264150944}
{'winner_predicted_by_total': 'Casper Stockham', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Casper Stockham', 'margin_unique': 0.6666666666666666}
{'winner_predicted_by_total': 'John Larson', 'margin_total': 0.9459459459459459, 'winner_predicted_by_unique': 'John Larson', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'not enough info', 'margin_total': nan, 'winner_predicted_by_un

{'winner_predicted_by_total': 'Vanessa Adia', 'margin_total': 0.4666666666666667, 'winner_predicted_by_unique': 'Vanessa Adia', 'margin_unique': 0.4166666666666667}
{'winner_predicted_by_total': 'Barbara Lee', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Barbara Lee', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Charlie Crist', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Charlie Crist', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'David Callahan', 'margin_total': nan, 'winner_predicted_by_unique': 'David Callahan', 'margin_unique': nan}
{'winner_predicted_by_total': 'Betsy Londrigan', 'margin_total': 0.9288537549407114, 'winner_predicted_by_unique': 'Betsy Londrigan', 'margin_unique': 0.9696969696969697}
{'winner_predicted_by_total': 'Rashida Tlaib', 'margin_total': nan, 'winner_predicted_by_unique': 'Rashida Tlaib', 'margin_unique': nan}
{'winner_predicted_by_total': 'Jineea Butler', 'margin_total': 0.8666666666666667, 'winner_predicted_by_unique': 'Jine

{'winner_predicted_by_total': 'Yvette Herrell', 'margin_total': 0.4117647058823529, 'winner_predicted_by_unique': 'Yvette Herrell', 'margin_unique': 0.48148148148148145}
{'winner_predicted_by_total': 'Liuba Grechen Shirley', 'margin_total': 0.9972770592239619, 'winner_predicted_by_unique': 'Liuba Grechen Shirley', 'margin_unique': 0.9965397923875432}
{'winner_predicted_by_total': 'Linda Coleman', 'margin_total': 0.9285714285714286, 'winner_predicted_by_unique': 'Linda Coleman', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Jill Schiller', 'margin_total': 0.92, 'winner_predicted_by_unique': 'Jill Schiller', 'margin_unique': 0.9090909090909091}
{'winner_predicted_by_total': 'Markwayne Mullin', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Markwayne Mullin', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Jamie McLeod-Skinner', 'margin_total': 0.863013698630137, 'winner_predicted_by_unique': 'Jamie McLeod-Skinner', 'margin_unique': 0.9183673469387755}
{'winner_predicted_

{'winner_predicted_by_total': 'Erik Paulsen', 'margin_total': 0.36140350877192984, 'winner_predicted_by_unique': 'Erik Paulsen', 'margin_unique': 0.13432835820895522}
{'winner_predicted_by_total': 'Michael Evans', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Michael Evans', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Adrian Smith', 'margin_total': 0.14285714285714285, 'winner_predicted_by_unique': 'Adrian Smith', 'margin_unique': 0.14285714285714285}
{'winner_predicted_by_total': 'Danny Tarkanian', 'margin_total': 0.964040536122916, 'winner_predicted_by_unique': 'Danny Tarkanian', 'margin_unique': 0.9694960212201591}
{'winner_predicted_by_total': 'Andrew Kim', 'margin_total': 0.5862068965517241, 'winner_predicted_by_unique': 'Andrew Kim', 'margin_unique': 0.6268656716417911}
{'winner_predicted_by_total': 'Ben Ray Lujan', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Ben Ray Lujan', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'not enough info', 'margin_tota

{'winner_predicted_by_total': 'Rodolfo Cortes Barragan', 'margin_total': 0.09090909090909091, 'winner_predicted_by_unique': 'Lucille Roybal-Allard', 'margin_unique': 0.0}
{'winner_predicted_by_total': 'Aja Smith', 'margin_total': 0.8503401360544217, 'winner_predicted_by_unique': 'Aja Smith', 'margin_unique': 0.7209302325581395}
{'winner_predicted_by_total': 'Julia Peacock', 'margin_total': 0.7567567567567568, 'winner_predicted_by_unique': 'Julia Peacock', 'margin_unique': 0.6}
{'winner_predicted_by_total': 'Omar Navarro', 'margin_total': 0.8098815371279977, 'winner_predicted_by_unique': 'Omar Navarro', 'margin_unique': 0.6967930029154519}
{'winner_predicted_by_total': 'Nanette Barragán', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Nanette Barragán', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Katie Porter', 'margin_total': 0.9489795918367347, 'winner_predicted_by_unique': 'Katie Porter', 'margin_unique': 0.9403973509933775}
{'winner_predicted_by_total': 'Lou Correa', '

{'winner_predicted_by_total': 'not enough info', 'margin_total': nan, 'winner_predicted_by_unique': 'not enough info', 'margin_unique': nan}
{'winner_predicted_by_total': 'Ed Perlmutter', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Ed Perlmutter', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Stephanie Murphy', 'margin_total': 0.5681818181818182, 'winner_predicted_by_unique': 'Stephanie Murphy', 'margin_unique': 0.6153846153846154}
{'winner_predicted_by_total': 'Carolyn Bourdeaux', 'margin_total': 0.9801980198019802, 'winner_predicted_by_unique': 'Carolyn Bourdeaux', 'margin_unique': 0.9574468085106383}
{'winner_predicted_by_total': 'Danny K. Davis', 'margin_total': nan, 'winner_predicted_by_unique': 'Danny K. Davis', 'margin_unique': nan}
{'winner_predicted_by_total': 'Andre Carson', 'margin_total': 1.0, 'winner_predicted_by_unique': 'Andre Carson', 'margin_unique': 1.0}
{'winner_predicted_by_total': 'Elijah Cummings', 'margin_total': 1.0, 'winner_predicted_by_unique': 

{'winner_predicted_by_total': 'Greg Gianforte', 'margin_total': 0.7547683923705722, 'winner_predicted_by_unique': 'Greg Gianforte', 'margin_unique': 0.8042553191489362}
{'winner_predicted_by_total': 'Matt Rosendale', 'margin_total': 0.8723613156602847, 'winner_predicted_by_unique': 'Matt Rosendale', 'margin_unique': 0.8199165175909362}
{'winner_predicted_by_total': 'Jane Raybould', 'margin_total': 0.9562043795620438, 'winner_predicted_by_unique': 'Jane Raybould', 'margin_unique': 0.96}
{'winner_predicted_by_total': 'Jacky Rosen', 'margin_total': 0.3209013209013209, 'winner_predicted_by_unique': 'Jacky Rosen', 'margin_unique': 0.2537704918032787}
{'winner_predicted_by_total': 'Chris Pappas', 'margin_total': 0.6451612903225806, 'winner_predicted_by_unique': 'Chris Pappas', 'margin_unique': 0.5675675675675675}
{'winner_predicted_by_total': 'Bob Hugin', 'margin_total': 0.8854916067146283, 'winner_predicted_by_unique': 'Bob Hugin', 'margin_unique': 0.8108108108108109}
{'winner_predicted_by_

In [59]:
new_df

,district,state,chamber,winner_predicted_by_total,winner_predicted_by_unique,margin_total,margin_unique
0,District 1,Alabama,house,Robert Kennedy Jr.,Robert Kennedy Jr.,0.280000,0.090909
1,District 1,Arizona,house,Wendy Rogers,Wendy Rogers,NaN,NaN
2,District 1,Arkansas,house,Rick Crawford,Elvis Presley,1.000000,NaN
3,District 1,California,house,Audrey Denney,Audrey Denney,0.891304,0.943396
4,District 1,Colorado,house,Casper Stockham,Casper Stockham,1.000000,0.666667
5,District 1,Connecticut,house,John Larson,John Larson,0.945946,1.000000
6,District 1,Georgia,house,not enough info,not enough info,NaN,NaN
7,District 1,Hawaii,house,Cam Cavasso,Cam Cavasso,1.000000,1.000000
8,District 1,Idaho,house,Russ Fulcher,Russ Fulcher,NaN,NaN
9,District 1,Illinois,house,Bobby Rush,Bobby Rush,1.000000,1.000000


In [61]:
new_df.to_csv('midterm_predictions.csv')

In [32]:
i

439

In [34]:
new_df[new_df['chamber'] == 'house'].shape

(406, 4)

In [240]:
rows

[{'chamber': 'house',
  'district': 'District 1',
  'state': 'Alabama',
  'winner': 'Robert Kennedy Jr.'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Arizona',
  'winner': 'Wendy Rogers'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Arkansas',
  'winner': 'Chintan Desai'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'California',
  'winner': 'Audrey Denney'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Colorado',
  'winner': 'Casper Stockham'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Connecticut',
  'winner': 'John Larson'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Georgia',
  'winner': 'Lisa Ring'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Hawaii',
  'winner': 'Cam Cavasso'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Idaho',
  'winner': 'Cristina McNeil'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Illinois',
  '

In [245]:
len(new_df[new_df["state"].isnull()])

43

In [20]:
with open('partisan_tweets_per_candidate_final.pkl', 'rb') as f:
    partisan_tweets_per_candidate = pickle.load(f)

In [21]:
partisan_tweets_per_candidate

{'kennedy4alabama': {'conservative': {'total': 0, 'unique': 0},
  'liberal': {'total': 32, 'unique': 18},
  'neutral': {'total': 227, 'unique': 129}},
 'repbyrne': {'conservative': {'total': 18, 'unique': 15},
  'liberal': {'total': 0, 'unique': 0},
  'neutral': {'total': 568, 'unique': 320}},
 'tabithak': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 86, 'unique': 61},
  'neutral': {'total': 809, 'unique': 412}},
 'repmartharoby': {'conservative': {'total': 3, 'unique': 1},
  'liberal': {'total': 4, 'unique': 1},
  'neutral': {'total': 446, 'unique': 246}},
 'haganforhouse': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 110, 'unique': 45},
  'neutral': {'total': 1726, 'unique': 721}},
 'repmikerogersal': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 3, 'unique': 2},
  'neutral': {'total': 277, 'unique': 150}},
 'leeauman2018': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 37, 'unique': 21},
  'neutral'

In [22]:
def get_winner(df, partisan_tweets_per_candidate):
    """
    return candidate to win under total, candidate to win 
    under unique, margin under total, margin under unique
    """
    results = {
        'winner_predicted_by_total': '', 
        'margin_total': 0, 
        'winner_predicted_by_unique':'', 
        'margin_unique': 0
    }
    metrics = ["total", "unique"]
    for metric in metrics:
        winner, margin = get_winner_by_metric(df, partisan_tweets_per_candidate, metric)
        results['winner_predicted_by_'+metric] = winner
        results['margin_'+metric] = margin
    return results


In [23]:
partisan_tweets_per_candidate['ann_kirkpatrick']['neutral']['total']

1364

In [57]:
def get_winner_by_metric(df, partisan_tweets_per_candidate, metric):
    # get data of top two candidates in number of total mentions
    num_of_neutral_tweets, score = {}, dict()
#     print(df['handles'])
    for handle in df['handles']:
        try:
#             print(handle, metric)
#             print(partisan_tweets_per_candidate[handle]['neutral'][metric])
#             if handle not in partisan_tweets_per_candidate:
#                 print(handle + " handle not in partisan_tweets_per_candidate")
#             if 'neutral' not in partisan_tweets_per_candidate[handle]:
#                 print('neutral not in partisan_tweets_per_candidate')
#             if metric not in partisan_tweets_per_candidate[handle]['neutral']:
#                 print(metric + " metric not in partisan_tweets_per_candidate[handle]['neutral']")
            num_of_neutral_tweets[handle] = partisan_tweets_per_candidate[handle]['neutral'][metric]
        except:
            pass
#     print(num_of_neutral_tweets)
    top_two_handles = sorted(num_of_neutral_tweets, key=num_of_neutral_tweets.get, reverse=True)
#     print(top_two_handles)
    affiliation_to_sentiment = {
        'Democratic Party': 'liberal',
        'Republican Party': 'conservative',
        'Libertarian Party': 'conservative',
        'Green Party': 'liberal'
    }
    if len(top_two_handles) < 2:
        return "not enough info", np.nan
    else:
        total_tweets = 0
        for i in range(2):
            handle = top_two_handles[i]
            affiliation = df.loc[df['handles'] == handle]['party'].values[0]
            try: 
                num_of_tweets = partisan_tweets_per_candidate[handle][affiliation_to_sentiment[affiliation]][metric] \
                    + partisan_tweets_per_candidate[handle][affiliation_to_sentiment[affiliation]][metric]
                score[handle] = num_of_tweets
                total_tweets += num_of_tweets
            except:
                pass
        try:
            margin = abs(score[top_two_handles[1]] - score[top_two_handles[0]]) / total_tweets
        except:
            margin = np.nan
        if score:
            return df.loc[df['handles'] == max(score, key=score.get)]['name'].values[0], margin
        else:
            return None, np.nan

In [115]:
partisan_tweets_per_candidate

{'kennedy4alabama': {'conservative': {'total': 0, 'unique': 0},
  'liberal': {'total': 32, 'unique': 18},
  'neutral': {'total': 227, 'unique': 129}},
 'repbyrne': {'conservative': {'total': 18, 'unique': 15},
  'liberal': {'total': 0, 'unique': 0},
  'neutral': {'total': 568, 'unique': 320}},
 'tabithak': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 86, 'unique': 61},
  'neutral': {'total': 809, 'unique': 412}},
 'repmartharoby': {'conservative': {'total': 3, 'unique': 1},
  'liberal': {'total': 4, 'unique': 1},
  'neutral': {'total': 446, 'unique': 246}},
 'haganforhouse': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 110, 'unique': 45},
  'neutral': {'total': 1726, 'unique': 721}},
 'repmikerogersal': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 3, 'unique': 2},
  'neutral': {'total': 277, 'unique': 150}},
 'leeauman2018': {'conservative': {'total': 1, 'unique': 1},
  'liberal': {'total': 37, 'unique': 21},
  'neutral'

We want the numerator to be the number of liberal/conservative tweets, the denominator will be the total number of tweets in the top two